# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

from timeit import default_timer as timer

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [5]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9708,HOBBIES_2_146_CA_4_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Hobbies'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [ ]:
len(sales_intermittent)

1301

In [ ]:
len(sales_lumpy)

395

In [ ]:
len(sales_erratic)

458

In [7]:
len(sales_smooth)

1061

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [8]:
sales_smooth

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9607,HOBBIES_2_045_CA_4_validation,HOBBIES_2_045,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9650,HOBBIES_2_088_CA_4_validation,HOBBIES_2_088,HOBBIES_2,HOBBIES,CA_4,CA,0,1,0,0,...,0,3,0,0,0,1,1,0,0,0
9651,HOBBIES_2_089_CA_4_validation,HOBBIES_2_089,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9671,HOBBIES_2_109_CA_4_validation,HOBBIES_2_109,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


# User-defined functions to calculate Metrics

In [9]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [10]:
sales_pattern = sales_smooth.copy()

In [11]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [12]:
dataframe_weekly

,wm_yr_wk,HOBBIES_1_001_CA_1,HOBBIES_1_001_CA_2,HOBBIES_1_001_CA_3,HOBBIES_1_001_CA_4,HOBBIES_1_004_CA_1,HOBBIES_1_004_CA_2,HOBBIES_1_004_CA_3,HOBBIES_1_004_CA_4,HOBBIES_1_005_CA_1,...,HOBBIES_2_115_CA_3,HOBBIES_2_115_CA_4,HOBBIES_2_117_CA_2,HOBBIES_2_120_CA_2,HOBBIES_2_121_CA_1,HOBBIES_2_121_CA_3,HOBBIES_2_128_CA_3,HOBBIES_2_132_CA_1,HOBBIES_2_139_CA_2,HOBBIES_2_147_CA_3
0,11101,4.0,3.0,4.0,1.0,18.0,23.0,34.0,4.0,3.0,...,6.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0
1,11102,3.0,6.0,5.0,3.0,16.0,23.0,45.0,1.0,6.0,...,4.0,5.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0
2,11103,5.0,4.0,7.0,4.0,13.0,15.0,46.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0
3,11104,7.0,11.0,6.0,2.0,5.0,38.0,47.0,7.0,2.0,...,2.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,5.0,2.0
4,11105,1.0,1.0,6.0,5.0,14.0,35.0,46.0,9.0,7.0,...,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,4.0,4.0,8.0,1.0,10.0,33.0,24.0,4.0,8.0,...,4.0,2.0,2.0,2.0,0.0,0.0,3.0,0.0,2.0,0.0
269,11609,4.0,0.0,5.0,4.0,12.0,20.0,59.0,6.0,9.0,...,10.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,2.0,2.0
270,11610,3.0,7.0,6.0,6.0,13.0,42.0,43.0,7.0,3.0,...,2.0,4.0,1.0,1.0,0.0,0.0,6.0,0.0,1.0,1.0
271,11611,3.0,1.0,5.0,4.0,11.0,31.0,40.0,7.0,8.0,...,5.0,2.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0


In [13]:
lags = list(range(1, 25))
moving_averages = list(range(1, 25)) 
stds = list(range(2, 25))
maximums = list(range(1, 25))
minimums = list(range(1, 25))
totals = list(range(1, 25))
num_days_zeros = list(range(1, 25))

correlation_threshold = 0.8

end_train_week = 11610

num_id_sample_for_tuning = 200

In [14]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [15]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [16]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [17]:
len(sample_list_filtered)

183

In [ ]:
start = timer()

df_output_hyper_params = pd.DataFrame()
    
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    dataframe_product = dataframe_weekly[['wm_yr_wk', product]]

    # Create Lag variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (t-{lag})': dataframe_product[product].shift(lag)
        for lag in lags
    })

    # Create Rolling Moving Average variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
        for moving_average in moving_averages
    })

    # Create Rolling Standard Deviation variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
        for std in stds
    })

    # Create Rolling Maximum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
        for maximum in maximums
    })

    # Create Rolling Minimum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
        for minimum in minimums
    })

    # Create Rolling Total variables 
    dataframe_product = dataframe_product.assign(**{
        f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
        for total in totals
    })


    dataframe_product.dropna(inplace=True)
    dataframe_product.set_index('wm_yr_wk', inplace=True)

    train_data = dataframe_product[dataframe_product.index <= end_train_week]

    corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
    high_corr_var=np.where(corr_matrix > correlation_threshold)
    high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

    train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

    X_train = train_data.drop(product, axis=1)
    y_train = train_data[product]  

    tscv = TimeSeriesSplit(n_splits=3)

    params = {'n_estimators': [30, 50],
              'max_depth': list(range(3,11)),
              'min_samples_leaf': range(4,11),
              'max_features': ['sqrt', 'log2', None],
              'bootstrap': [True], #We force bootstrap
              'max_samples': [0.6, 0.7, 0.8, 0.9]}

    forest = RandomForestRegressor(n_jobs=1)

    regressor = RandomizedSearchCV(estimator=forest,
                                  param_distributions=params,
                                  cv = tscv, 
                                  scoring='neg_mean_absolute_percentage_error',
                                  n_iter=150,
                                  verbose=1)
    regressor.fit(X_train, y_train)
    best_params_temp = regressor.best_params_

    df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
    df_output_temp['Product'] = product

    df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)

    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_3_196_CA_1
Progressing: 0.0 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_082_CA_3
Progressing: 0.67 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_1_147_CA_2
Progressing: 1.34 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_250_CA_3
Progressing: 2.01 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_710_CA_3
Progressing: 2.68 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_298_CA_1
Progressing: 3.36 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_304_CA_4
Progressing: 4.03 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_2_036_CA_3
Progressing: 4.7 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_2_051_CA_3
Progressing: 5

In [ ]:
df_output_hyper_params.to_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/Random Forest (Completed Pipeline)/Foods/Smooth_Params_Tuning_Random Forest.csv')

# Decide optimum set of parameters for Random Forest algorithm

In [ ]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_output_hyper_params.groupby(['n_estimators', 'max_depth', 'min_samples_leaf', 'max_features', 'max_samples'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
n_estimators max_depth min_samples_leaf max_features max_samples         
30           4         6                log2         0.7                2
             3         6                sqrt         0.9                1
             7         7                log2         0.9                1
50           8         5                sqrt         0.9                1
             4         9                log2         0.6                1
30           10        10               sqrt         0.9                1
                       7                log2         0.6                1
                       6                sqrt         0.8                1
                       4                sqrt         0.8                1
             9         10               log2         0.8                1
                       8                sqrt         0.9                1
                       4                log2         0.9                1
             8         9                log2         0.9                1
                       4                log2         0.7                1
             7         10               log2         0.6                1
                       5                log2         0.9                1
             3         7                sqrt         0.6                1
             7         4                sqrt         0.9                1
             6         8                log2         0.8                1
                       7                sqrt         0.6                1
                       6                log2         0.7                1
                       5                sqrt         0.8                1
                                        log2         0.8                1
             5         8                log2         0.7                1
                       4                sqrt         0.9                1
             4         6                sqrt         0.9                1
                       5                sqrt         0.6                1
                       4                sqrt         0.8                1
                                                     0.6                1
50           10        9                log2         0.9                1

In [ ]:
df_output_hyper_params.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
30,111
50,38


In [ ]:
df_output_hyper_params.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
3,10
4,26
5,18
6,21
7,19
8,21
9,13
10,21


In [ ]:
# See what value of parameter "max_depth" occurs the most frequently
df_output_hyper_params.groupby(['min_samples_leaf'])[['Product']].count()

,Product
min_samples_leaf,
4,43
5,24
6,27
7,21
8,12
9,10
10,12


In [ ]:
# See what value of parameter "learning_rate" occurs the most frequently
df_output_hyper_params.groupby(['max_features'])[['Product']].count()

,Product
max_features,
log2,16
sqrt,15


In [ ]:
# See what value of parameter "colsample_bytree" occurs the most frequently
df_output_hyper_params.groupby(['max_samples'])[['Product']].count()

,Product
max_samples,
0.6,33
0.7,20
0.8,34
0.9,62


# Implement Random Forest on Test Set with optimum set of parameters

### Specify optimum set of parameters

In [18]:
n_estimators_chosen = 30
max_depth_chosen = 4
min_samples_leaf_chosen = 6
max_features_chosen = 'log2'
max_samples_chosen = 0.7

In [19]:
len(list_pattern_py)

1061

In [20]:
list_pattern_temp = list_pattern_py.copy()

In [21]:
start = timer()

df_output_rf_pattern = pd.DataFrame()
for product in list_pattern_temp:
  print('Currently Running: %s' % product)
  print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))
  dataframe_product = dataframe_weekly[['wm_yr_wk', product]]
  # Create Lag variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (t-{lag})': dataframe_product[product].shift(lag)
      for lag in lags
  })

  # Create Rolling Moving Average variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
      for moving_average in moving_averages
  })

  # Create Rolling Standard Deviation variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
      for std in stds
  })

  # Create Rolling Maximum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
      for maximum in maximums
  })

  # Create Rolling Minimum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
      for minimum in minimums
  })

  # Create Rolling Total variables 
  dataframe_product = dataframe_product.assign(**{
      f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
      for total in totals
  })

  dataframe_product.dropna(inplace=True)
  dataframe_product.set_index('wm_yr_wk', inplace=True)

  train_data = dataframe_product[dataframe_product.index <= end_train_week]
  test_data = dataframe_product[dataframe_product.index > end_train_week]

  corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
  high_corr_var=np.where(corr_matrix > correlation_threshold)
  high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

  train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
  test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

  X_train = train_data.drop(product, axis=1)
  y_train = train_data[product]

  X_test = test_data.drop(product, axis=1)
  y_test = test_data[product]

  forest = RandomForestRegressor(n_jobs=1,
                                n_estimators = n_estimators_chosen,
                                min_samples_leaf = min_samples_leaf_chosen,
                                max_samples = max_samples_chosen,
                                max_features = max_features_chosen,
                                max_depth = max_depth_chosen,
                                bootstrap = [True],
                                random_state = 0)
  

  forest.fit(X_train,y_train)
  y_pred = forest.predict(X_test)
  df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                  'Forecast': y_pred, 
                                  'Product': [product for num_rows in range(len(y_pred))]
                                  })
  
  df_output_rf_pattern = df_output_rf_pattern.append(df_output_temp, ignore_index=False)  

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOBBIES_1_001_CA_1
Progressing: 0.0 %
Currently Running: HOBBIES_1_004_CA_1
Progressing: 0.09 %
Currently Running: HOBBIES_1_005_CA_1
Progressing: 0.19 %
Currently Running: HOBBIES_1_006_CA_1
Progressing: 0.28 %
Currently Running: HOBBIES_1_007_CA_1
Progressing: 0.38 %
Currently Running: HOBBIES_1_008_CA_1
Progressing: 0.47 %
Currently Running: HOBBIES_1_009_CA_1
Progressing: 0.57 %
Currently Running: HOBBIES_1_010_CA_1
Progressing: 0.66 %
Currently Running: HOBBIES_1_012_CA_1
Progressing: 0.75 %
Currently Running: HOBBIES_1_013_CA_1
Progressing: 0.85 %
Currently Running: HOBBIES_1_014_CA_1
Progressing: 0.94 %
Currently Running: HOBBIES_1_015_CA_1
Progressing: 1.04 %
Currently Running: HOBBIES_1_016_CA_1
Progressing: 1.13 %
Currently Running: HOBBIES_1_017_CA_1
Progressing: 1.23 %
Currently Running: HOBBIES_1_019_CA_1
Progressing: 1.32 %
Currently Running: HOBBIES_1_020_CA_1
Progressing: 1.41 %
Currently Running: HOBBIES_1_021_CA_1
Progressing: 1.51 %
Currently Runni

In [22]:
df_output_rf_pattern

,Actual Data,Forecast,Product
wm_yr_wk,,,
11611,3.0,3.378959,HOBBIES_1_001_CA_1
11612,1.0,3.459930,HOBBIES_1_001_CA_1
11611,11.0,11.272646,HOBBIES_1_004_CA_1
11612,15.0,10.635945,HOBBIES_1_004_CA_1
11611,8.0,5.963827,HOBBIES_1_005_CA_1
...,...,...,...
11612,3.0,2.362467,HOBBIES_2_089_CA_4
11611,0.0,0.046555,HOBBIES_2_109_CA_4
11612,0.0,0.046555,HOBBIES_2_109_CA_4


In [23]:
df_output_rf_pattern.to_csv('Smooth_Random_Forest_Forecast_Values.csv')

In [24]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [25]:
# Start calculating metrics 
start = timer()

df_forecast_metrics = df_output_rf_pattern.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.03300786011666711 minutes


In [26]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_001_CA_1,0.709722,70.9722,61.0970,129.3125,1.419445,1.759953
1,HOBBIES_1_001_CA_2,0.436904,58.2539,65.8420,69.2891,1.747617,1.960825
2,HOBBIES_1_001_CA_3,0.572224,50.8644,55.5577,39.1865,4.577795,6.034791
3,HOBBIES_1_001_CA_4,0.417882,27.8588,29.4234,37.8414,0.835764,0.984337
4,HOBBIES_1_004_CA_1,0.579588,17.8335,18.2473,15.7862,2.318351,3.091870
...,...,...,...,...,...,...,...
1056,HOBBIES_2_121_CA_3,inf,inf,200.0000,NaN,0.103982,0.103982
1057,HOBBIES_2_128_CA_3,0.650947,55.7955,60.6082,55.4009,1.952841,2.135127
1058,HOBBIES_2_132_CA_1,inf,inf,200.0000,NaN,0.146188,0.146188
1059,HOBBIES_2_139_CA_2,0.546939,109.3877,125.9112,41.1579,1.093877,1.126879


In [27]:
df_forecast_metrics.to_csv('Random_Forest_Smooth_Forecast_Metrics.csv')